In [102]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

# import decision tree classifier
from sklearn.tree import DecisionTreeClassifier




In [104]:
data = pd.read_csv('output_variables_1.csv')
data
X = data.drop('Price', axis=1)
y = data[['Price']]

# do fitting
reg = LinearRegression().fit(X, y)
reg.score(X, y)
# save reg to pickle
import pickle
with open('reg.pkl', 'wb') as f:
    pickle.dump(reg, f)
    

# open reg from pickle
with open('reg.pkl', 'rb') as f:
    reg = pickle.load(f)
    
# reg predict first row
reg.predict(X.iloc[0:1, :])[0][0]
# show the coefficients and create dataframe with each of the feature name as header
coefficients = reg.coef_
coefficients = coefficients[0]
coefficients = pd.DataFrame(coefficients, index=X.columns, columns=['Coefficient'])
coefficients

,Coefficient
Net Sales,-0.178422
COGS,0.217548
Trading Income,0.000000
Other Income,-0.003289
Interest Revenue of Bank,-0.002057
Advertising & Promotion Expense,0.003691
Operational Expense,0.004847
Other Expense,-0.362341
Depreciation of Equipment Clinic Expense,-0.063470
Depreciation of Equipment Non Clinic Expense,0.000000


In [91]:
data = pd.read_csv('output_variables_1.csv')
data

,Net Sales,COGS,Trading Income,Other Income,Interest Revenue of Bank,Advertising & Promotion Expense,Operational Expense,Other Expense,Depreciation of Equipment Clinic Expense,Depreciation of Equipment Non Clinic Expense,Bank Tax Expense,Other Tax,Tangible Assets (PP&E),Net Cash Flow Average,Net Cash Flow Standard Deviation,Net Cash Flow Trend Coefficient,Equipment Usage Ratio,Total Equipments,Price
0,1862463.95,-1053087.000,0.0,6932.24,4334.5,-8064.0,-10216.0,-311702.0,0.0000,0.0,-1547.0,0.00,200000.0,68168.166667,11140.327699,-2260.559441,0.5,8,5573000
1,1486403.39,-594561.356,0.0,0.00,0.0,-285.0,0.0,-1075407.5,-133776.3051,0.0,0.0,-76677.75,200000.0,76198.416667,8229.738651,482.793706,0.5,8,6028000


In [105]:
import pandas as pd
import numpy as np
from scipy.optimize import nnls
import pickle

# Load the data
data = pd.read_csv('output_variables_1.csv')
X = data.drop('Price', axis=1).values  # Convert to NumPy array for nnls
y = data['Price'].values  # Convert to 1D NumPy array

# Perform Non-Negative Least Squares fitting
coefficients, _ = nnls(X, y)

# Create a DataFrame with feature names and coefficients
coefficients_df = pd.DataFrame(coefficients, index=data.drop('Price', axis=1).columns, columns=['Coefficient'])

# Save the coefficients to a pickle file
with open('nnls_coefficients.pkl', 'wb') as f:
    pickle.dump(coefficients_df, f)

# Load the coefficients from the pickle file
with open('nnls_coefficients.pkl', 'rb') as f:
    coefficients_df = pickle.load(f)

# Predict the first row using the fitted NNLS coefficients
first_row_prediction = np.dot(X[0, :], coefficients)

# Display the prediction for the first row and the coefficients DataFrame
print(f'Prediction for the first row: {first_row_prediction}')
print(coefficients_df)


Prediction for the first row: 5572999.999999995
                                              Coefficient
Net Sales                                        6.626121
COGS                                             6.426735
Trading Income                                   0.000000
Other Income                                     0.000000
Interest Revenue of Bank                         0.000000
Advertising & Promotion Expense                  0.000000
Operational Expense                              0.000000
Other Expense                                    0.000000
Depreciation of Equipment Clinic Expense         0.000000
Depreciation of Equipment Non Clinic Expense     0.000000
Bank Tax Expense                                 0.000000
Other Tax                                        0.000000
Tangible Assets (PP&E)                           0.000000
Net Cash Flow Average                            0.000000
Net Cash Flow Standard Deviation                 0.000000
Net Cash Flow Trend Coef

In [107]:
import pandas as pd
import numpy as np
import cvxpy as cp
import pickle

# Load the data
data = pd.read_csv('output_variables_1.csv')
X = data.drop('Price', axis=1).values  # Convert to NumPy array for cvxpy
y = data['Price'].values  # Convert to 1D NumPy array

# Define the coefficients and intercept as variables
beta = cp.Variable(X.shape[1])
intercept = cp.Variable()

# Define the objective function (minimize the least squares error)
objective = cp.Minimize(cp.sum_squares(X @ beta + intercept - y))

# Define constraints: ensure all coefficients are non-negative
constraints = [beta >= 0]

# Create and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Extract the coefficients and intercept
coefficients = beta.value
intercept_value = intercept.value

# Create a DataFrame with feature names and coefficients
coefficients_df = pd.DataFrame(coefficients, index=data.drop('Price', axis=1).columns, columns=['Coefficient'])
coefficients_df['Intercept'] = intercept_value

# Save the coefficients to a pickle file
with open('cvxpy_coefficients_non_negative.pkl', 'wb') as f:
    pickle.dump(coefficients_df, f)

# Load the coefficients from the pickle file (for testing purposes)
with open('cvxpy_coefficients_non_negative.pkl', 'rb') as f:
    coefficients_df = pickle.load(f)

# Display the coefficients DataFrame
print(coefficients_df)


                                              Coefficient     Intercept
Net Sales                                        0.027496  6.867645e+06
COGS                                             1.063607  6.867645e+06
Trading Income                                   0.000000  6.867645e+06
Other Income                                     1.322544  6.867645e+06
Interest Revenue of Bank                         1.322544  6.867645e+06
Advertising & Promotion Expense                 13.111318  6.867645e+06
Operational Expense                             13.737705  6.867645e+06
Other Expense                                    0.117528  6.867645e+06
Depreciation of Equipment Clinic Expense         1.022396  6.867645e+06
Depreciation of Equipment Non Clinic Expense     0.000000  6.867645e+06
Bank Tax Expense                                13.591698  6.867645e+06
Other Tax                                        1.301627  6.867645e+06
Tangible Assets (PP&E)                           0.000801  6.867

In [108]:
import pandas as pd
import itertools

# Define the variables
EBIT = [100000, 254786, 400000]
EBIT_Ratio = [0, 0.1, 0.22, 0.3, 0.4]
Net_Sales_Growth = [0, 0.05, 0.1, 0.2, 0.3]

# Generate all combinations
combinations = list(itertools.product(EBIT, EBIT_Ratio, Net_Sales_Growth))

# Create a DataFrame from the combinations
df = pd.DataFrame(combinations, columns=['EBIT', 'EBIT Ratio', 'Net Sales Growth'])

df.to_csv('combinations.csv', index=False)

In [118]:
data = pd.read_csv('EBIT_baseline_to_multiple.csv')

In [119]:


# Define the what-if function using the loaded data
def what_if_ebit(ebit, ebit_ratio, net_sales_growth):
    # Define boundary values based on the unique values in the dataset
    ebit_boundaries = sorted(data['EBIT'].unique())
    ebit_ratio_boundaries = sorted(data['EBIT Ratio'].unique())
    net_sales_growth_boundaries = sorted(data['Net Sales Growth'].unique())

    # Round down to nearest boundary value
    ebit = max([val for val in ebit_boundaries if val <= ebit])
    ebit_ratio = max([val for val in ebit_ratio_boundaries if val <= ebit_ratio])
    net_sales_growth = max([val for val in net_sales_growth_boundaries if val <= net_sales_growth])

    # Lookup the row with the closest match
    result = data[(data['EBIT'] == ebit) & 
                  (data['EBIT Ratio'] == ebit_ratio) & 
                  (data['Net Sales Growth'] == net_sales_growth)]

    # Return the EBIT Multiple if a match is found, else return None
    if not result.empty:
        return result['EBIT Multiple'].values[0]
    else:
        return None

# Example usage of the function with user-specified input
example_ebit = 200000
example_ebit_ratio = 0.15
example_net_sales_growth = 0.1

what_if_result = what_if_ebit(example_ebit, example_ebit_ratio, example_net_sales_growth)
what_if_result


1.05

In [120]:
# Example usage of the function
example_ebit = 400000
example_ebit_ratio = 0.05
example_net_sales_growth = 0.5

what_if_result = what_if_ebit(example_ebit, example_ebit_ratio, example_net_sales_growth)
what_if_result

4.95

In [1]:
import pandas as pd

data_df = pd.DataFrame(
    {
        "Equipment": [
        "Intra Oral Camera",
        "Bleaching Unit",
        "Ultrasonic Scaler",
        "Light Cure",
        "Dental Unit",
        "Portable Xrays",
        "Endomotor",
        "Autoclaves",
        "Ultrasonic Cleaner",
        "Water Tank",
        "Prophylaxis Hand Piece",
        "Handpiece Set",
        "Compressor",
        "Apex Locator",
        "Dental Loupe",
        "Portable Light",
        "Camera DSLR",
        "Water Tank Hose",
        "Sealing Machine",
        "Xray Sensor"
    ],
        "Own?": [True] * 20,  # Default value for ownership is True
        "Quantity": [1] * 20 ,  # Random integer values between 1 and 4,
        "Expected Lifetime": [4] * 20,  # Random integer values between 3 and 7
        "Current Lifetime Usage": [2] * 20  # Initialize with zeros
    }
)

In [2]:
# add columns to the dataframe called price
data_df['Price'] = [1729.5,
487.5,
1349.5,
595,
30500,
9750,
997.5,
11500,
2500,
900,
175,
2000,
3325,
600,
250,
250,
2907.5,
10,
1500,
9500]

data_df.to_csv('equipment_data.csv', index=False)

In [ ]:
import pandas as pd
import itertools

# Define the variables
Number_of_Dentist = [1, 2, 3, 4, 5]
EBIT_Ratio = [0, 0.1, 0.22, 0.3, 0.4]
Net_Sales_Growth = [0, 0.05, 0.1, 0.2, 0.3]

# Generate all combinations
combinations = list(itertools.product(EBIT, EBIT_Ratio, Net_Sales_Growth))

# Create a DataFrame from the combinations
df = pd.DataFrame(combinations, columns=['EBIT', 'EBIT Ratio', 'Net Sales Growth'])

df.to_csv('combinations_dentist_count.csv', index=False)

In [1]:
import pandas as pd
import numpy as np

# Manually defining the starting point to ensure both datasets have equal cumulative sales over 36 months.

# Recalculate monthly and annual increase factors for clarity
months = 36  # Total number of months in the dataset
annual_increase_rate = 0.10  # 10% annual increase
monthly_increase_factor = (1 + annual_increase_rate) ** (1 / 12)  # Convert annual increase to a monthly factor
annual_increase_factor = 1 + annual_increase_rate  # Annual increase factor remains 10% for the second dataset

# Start with a reasonable base sales amount and adjust it
starting_sales = 1000  # Starting guess for a common base sales amount

# Calculate monthly sales with compound monthly growth
monthly_sales = [starting_sales * (monthly_increase_factor ** i) for i in range(months)]

# Calculate annual sales where values remain the same for each year
annual_sales = []
for i in range(months):
    year = i // 12  # Determine the current year (0, 1, 2)
    sales_value = starting_sales * (annual_increase_factor ** year)
    annual_sales.append(sales_value)

# Calculate the cumulative totals for both datasets
total_monthly_sales = sum(monthly_sales)
total_annual_sales = sum(annual_sales)

# Adjust the starting sales amount proportionally to match cumulative totals
scaling_factor = total_annual_sales / total_monthly_sales
adjusted_starting_sales = starting_sales * scaling_factor

# Recalculate the sales datasets with the adjusted starting value
adjusted_monthly_sales = [adjusted_starting_sales * (monthly_increase_factor ** i) for i in range(months)]
adjusted_annual_sales = []
for i in range(months):
    year = i // 12
    sales_value = adjusted_starting_sales * (annual_increase_factor ** year)
    adjusted_annual_sales.append(sales_value)

# Create the final DataFrame for display
final_adjusted_data = {
    'Month': pd.date_range(start='2024-01-01', periods=months, freq='M'),
    'Monthly_Increase_Sales': adjusted_monthly_sales,
    'Annual_Increase_Sales': adjusted_annual_sales
}

df_final_adjusted = pd.DataFrame(final_adjusted_data)

In [5]:
df_final_adjusted

,Month,Monthly_Increase_Sales,Annual_Increase_Sales
0,2024-01-31,956.896851,956.896851
1,2024-02-29,964.527281,956.896851
2,2024-03-31,972.218557,956.896851
3,2024-04-30,979.971165,956.896851
4,2024-05-31,987.785592,956.896851
5,2024-06-30,995.662333,956.896851
6,2024-07-31,1003.601885,956.896851
7,2024-08-31,1011.604747,956.896851
8,2024-09-30,1019.671425,956.896851
9,2024-10-31,1027.802428,956.896851


In [6]:
# do linear regression for column Monthly_Increase_Sales and return the slope
from sklearn.linear_model import LinearRegression

X = np.arange(len(df_final_adjusted)).reshape(-1, 1)
y = df_final_adjusted['Monthly_Increase_Sales']
y = df_final_adjusted['Annual_Increase_Sales']

reg = LinearRegression().fit(X, y)
reg.score(X, y) 
slope = reg.coef_[0]
slope


7.448278195213878

### Preprocess Haoey Data

In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('haoey_data.csv')
data

,Provider,Patient,No,Date,Item,Treatment Code Name Original,Tooth,Note,Invoice,Payment,Amount,Location,year,date1,PurchaseDate,Total_cost
0,"Ha, Thomas","Jones, Kim",1750301689,2023-12-21 00:00:00,72,Photographic records – intraoral – per appoint...,NaN,No,117828.0,Good,30.0,Potts Point,2023,2023-12-21 00:00:00,2023-12-21,30.0
1,"Ha, Thomas","Jones, Kim",1750301689,2023-12-21 00:00:00,72,Photographic records – intraoral – per appoint...,NaN,No,117828.0,Good,30.0,Potts Point,2023,2023-12-21 00:00:00,2023-12-21,30.0
2,"Ha, Thomas","Jones, Kim",1750301689,2023-12-21 00:00:00,114,Cleaning,NaN,No,117828.0,Good,160.0,Potts Point,2023,2023-12-21 00:00:00,2023-12-21,160.0
3,"Ha, Thomas","Jones, Kim",1750301689,2023-12-21 00:00:00,11,Oral Examination,NaN,Yes,117828.0,Good,110.0,Potts Point,2023,2023-12-21 00:00:00,2023-12-21,110.0
4,"Ha, Thomas","Shaw, Jeannette",2-000713,2023-12-21 00:00:00,613,Crown,27.0,No,117827.0,Good,1800.0,Potts Point,2023,2023-12-21 00:00:00,2023-12-21,1800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31958,"Oey, Julian","Ha, Steven",1-005067,2021-01-04 00:00:00,114,Cleaning,NaN,Yes,72099.0,Good,140.0,Matraville,2021,2021-01-04 00:00:00,2021-01-04,140.0
31959,"Oey, Julian","Ha, Steven",1-005067,2021-01-04 00:00:00,121,Remineralisation Agents,NaN,Yes,72099.0,Good,60.0,Matraville,2021,2021-01-04 00:00:00,2021-01-04,60.0
31960,"Oey, Julian","Ha, Steven",1-005067,2021-01-04 00:00:00,72,Photographic records – intraoral – per appoint...,18.0,Yes,72099.0,Good,40.0,Matraville,2021,2021-01-04 00:00:00,2021-01-04,40.0
31961,"Oey, Julian","Ha, Steven",1-005067,2021-01-04 00:00:00,73,Photographic records – extraoral – per appoint...,18.0,Yes,72099.0,Good,40.0,Matraville,2021,2021-01-04 00:00:00,2021-01-04,40.0


In [13]:
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data_by_year = data.groupby('Year')['Patient'].nunique()
data_by_year


Year
2021    1021
2022    1109
2023    1005
Name: Patient, dtype: int64

In [14]:
data_by_year.mean()

1045.0

In [3]:
import pandas as pd
import numpy as np

data = pd.read_excel(r'company_data/Acre Data.xlsx', sheet_name='main_variables')
data

,Category,Sub Category,Variable,Value
0,Profit & Loss,Gross Profit,Net Sales,1.862464e+06
1,Profit & Loss,Gross Profit,COGS,-1.053087e+06
2,Profit & Loss,Gross Profit,Trading Income,0.000000e+00
3,Profit & Loss,Other Income,Other Income,6.932240e+03
4,Profit & Loss,Other Income,Interest Revenue of Bank,-4.334500e+03
5,Profit & Loss,Operating Expense,Advertising & Promotion Expense,-8.063710e+03
6,Profit & Loss,Operating Expense,Operational Expense,-1.021609e+04
7,Profit & Loss,Operating Expense,Other Expense,-3.117025e+05
8,Profit & Loss,Depreciation,Depreciation of Equipment Clinic Expense,0.000000e+00
9,Profit & Loss,Depreciation,Depreciation of Equipment Non Clinic Expense,0.000000e+00


In [5]:
# convert the last three row of 'Value' column to integer
data['Value'].iloc[-3:] = data['Value'].iloc[-3:].astype(int)

C:\Users\Zak\AppData\Local\Temp\ipykernel_14764\2904961405.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Value'].iloc[-3:] = data['Value'].iloc[-3:].astype(int)


In [ ]:
d